In [1]:
from torch import nn
from torch.nn import functional as F
import torch

残差神经网络思想：只有当较复杂的函数类包含较小的函数类时，我们才能确保提高它们的性能。 对于深度神经网络，如果我们能将新添加的层训练成恒等映射（identity function）
，新模型和原模型将同样有效。 同时，由于新模型可能得出更优的解来拟合训练数据集，因此添加层似乎更容易降低训练误差。

 残差网络的核心思想是：每个附加层都应该更容易地包含原始函数作为其元素之一。 于是，残差块（residual blocks）便诞生了，这个设计对如何建立深层神经网络产生了深远的影响。

In [2]:
class Residual(nn.Module):
    def __init__(self, input_channels, num_channels,use_1x1conv=False,strides=1):
        super(Residual, self).__init__()
        self.conv1=nn.Conv2d(in_channels=input_channels,out_channels=num_channels,kernel_size=3,stride=strides,padding=1)
        self.conv2=nn.Conv2d(in_channels=num_channels,out_channels=num_channels,kernel_size=3,stride=1,padding=1)
        self.relu=nn.ReLU(inplace=True)
        if use_1x1conv:
            ##大小必须要跟上面的一样 因此 strides要继承
            self.conv3=nn.Conv2d(in_channels=input_channels,out_channels=num_channels,kernel_size=3,stride=strides,padding=1)
        else:
            self.conv3=None
        self.bn1=nn.BatchNorm2d(num_features=num_channels)
        self.bn2=nn.BatchNorm2d(num_features=num_channels)
    def forward(self, x):
        y=F.relu(self.bn1(self.conv1(x)))
        y=self.bn2(self.conv2(y))
        if self.conv3:
            x=self.conv3(x)
        y=y+x
        return self.relu(y)
        

In [3]:
blk=Residual(3,3)

In [4]:
x=torch.rand(4,3,6,6)
y=blk(x)

In [5]:
y.shape

torch.Size([4, 3, 6, 6])

In [38]:
b1=nn.Sequential(nn.Conv2d(1,64,kernel_size=7,stride=2,padding=3),nn.BatchNorm2d(64),nn.ReLU()
                 ,nn.MaxPool2d(kernel_size=3,stride=2,padding=1))

def resnet_block(in_channels,out_channels,num_residuals,first_block=False):
    blk=[]
    for i in range(num_residuals):
        if i==0 and not first_block:
            blk.append(Residual(in_channels,out_channels,use_1x1conv=True,strides=2))
        else:
            blk.append(Residual(out_channels,out_channels))
    return blk

In [39]:
b2 = nn.Sequential(*resnet_block(64, 64, 2, first_block=True))
b3 = nn.Sequential(*resnet_block(64, 128, 2))
b4 = nn.Sequential(*resnet_block(128, 256, 2))
b5 = nn.Sequential(*resnet_block(256, 512, 2))
net=nn.Sequential(b1,b2,b3,b4,b5,nn.AdaptiveAvgPool2d(1),nn.Flatten(),nn.Linear(512,10))

In [40]:
X = torch.rand(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 128, 28, 28])
Sequential output shape:	 torch.Size([1, 256, 14, 14])
Sequential output shape:	 torch.Size([1, 512, 7, 7])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 512, 1, 1])
Flatten output shape:	 torch.Size([1, 512])
Linear output shape:	 torch.Size([1, 10])


# 密集神经网络densenet

In [1]:
#DenseNet使用了ResNet改良版的“批量规范化、激活和卷积”架构（参见 7.6节中的练习）。 我们首先实现一下这个架构。
import torch
from torch import nn
def conv_block(input_channels,out_channels):
    return nn.Sequential(nn.BatchNorm2d(input_channels),nn.ReLU(),nn.Conv2d(input_channels,out_channels,kernel_size=3,padding=1))

In [13]:
##稠密块
class DenseBlock(nn.Module):
    def __init__(self, num_convs,input_channels,num_channels):
        super().__init__()
        layer=[]
        for i in range(num_convs):
            layer.append(conv_block(input_channels+num_channels*i,num_channels))
        self.net=nn.Sequential(*layer)
    def forward(self,x):
        for blk in self.net:
            y=blk(x)
            x=torch.cat((x,y),dim=1)
        return x

In [14]:
blk = DenseBlock(2, 3, 10)
X = torch.randn(2, 3, 8, 8)
Y = blk(X)
Y.shape

In [16]:
##过渡层 用来缩减通道数
def transition_block(input_channels,out_channels):
    return nn.Sequential(
        nn.BatchNorm2d(input_channels),nn.ReLU()
         ,nn.Conv2d(input_channels,out_channels,kernel_size=1)
         ,nn.AvgPool2d(kernel_size=2,stride=2)
    )

In [17]:
blk = transition_block(23, 10)
blk(Y).shape

torch.Size([2, 10, 4, 4])

In [19]:
## densenet模型
b1=nn.Sequential(nn.Conv2d(1,64,kernel_size=7,stride=2,padding=3),nn.BatchNorm2d(64),nn.ReLU()
                 ,nn.BatchNorm2d(64),nn.ReLU(),nn.MaxPool2d(kernel_size=3,stride=2,padding=1))

In [20]:
# num_channels为当前的通道数
num_channels, growth_rate = 64, 32
num_convs_in_dense_blocks = [4, 4, 4, 4]
blks = []
for i, num_convs in enumerate(num_convs_in_dense_blocks):
    blks.append(DenseBlock(num_convs, num_channels, growth_rate))
    # 上一个稠密块的输出通道数
    num_channels += num_convs * growth_rate
    # 在稠密块之间添加一个转换层，使通道数量减半
    if i != len(num_convs_in_dense_blocks) - 1:
        blks.append(transition_block(num_channels, num_channels // 2))
        num_channels = num_channels // 2

In [22]:
net = nn.Sequential(
    b1, *blks,
    nn.BatchNorm2d(num_channels), nn.ReLU(),
    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten(),
    nn.Linear(num_channels, 10))

In [23]:
x=torch.rand(size=(1, 1, 224, 224))

In [24]:
net(x)

tensor([[-0.0267,  0.1980,  0.1214,  0.1914, -0.0412,  0.2882,  0.0563, -0.1331,
          0.0140,  0.1392]], grad_fn=<AddmmBackward0>)